<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f2a65441aef689fa232b670b9ce38bf1881f36801cd0e56e4d8ca3be00240c27
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 10:40:46
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 12.37 K (0.09%)
Current PnL: -17.60 L (-11.84%)
CY Booked + Current PnL: -6.57 L (-4.42%)
-------------------
Total profit:  2.05 L
Total loss:  -19.65 L
-------------------
Total Booked + Current PnL: 20.41 L (16.77%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.70 L (59.11%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.42,59.0,H-LC,11.17,172964.0,16787.0,9305.0,-0.22,10.75,5.38,16.71,22.0,1.80,1.26,30.06,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,60.0,M-LC,4.27,218148.0,16668.0,10537.0,-0.05,8.27,4.83,13.51,60.0,1.58,1.59,21.91,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,100.85,51.0,M-SC,5.02,86532.0,-14245.0,14338.0,-1.87,-14.14,16.57,0.09,245.0,-0.99,0.63,13.68,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.23,114597.0,14093.0,16548.0,0.85,14.02,14.44,30.49,79.0,0.85,0.83,50.97,MH,ATH,METALS
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.57,39.0,H-MC,2.09,135046.0,-742.0,43498.0,-0.15,-0.55,32.21,31.48,107.0,-0.02,0.98,11.51,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.68,245638.0,-2093.0,77155.0,0.01,-0.84,31.41,30.30,92.0,-0.03,1.79,12.33,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-31.21,49.0,H-MC,4.54,198945.0,-1875.0,70645.0,-0.32,-0.93,35.51,34.25,80.0,-0.03,1.45,4.21,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.58,38.0,H-LC,6.04,300689.0,-15153.0,127823.0,-0.71,-4.80,42.51,35.67,5.0,-0.12,2.19,8.39,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.52,267761.0,396.0,65066.0,0.21,0.15,24.30,24.48,48.0,0.01,1.95,55.20,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.06,225165.0,-8979.0,57597.0,-0.75,-3.83,25.58,20.77,86.0,-0.16,1.64,2.82,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,230052.0,-11872.0,75135.0,-0.16,-4.91,32.66,26.15,8.0,-0.16,1.67,8.71,X40,ATH,IT
56,RELIANCE,1533.00,-13.29,51.0,H-LC,6.04,216824.0,6458.0,22333.0,0.48,3.07,10.30,13.68,37.0,0.29,1.58,20.10,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,59.0,H-LC,1.19,238792.0,-7359.0,86562.0,0.28,-2.99,36.25,32.18,16.0,-0.09,1.74,31.98,X200,ATH,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.84,60.0,H-LC,3.70,283491.0,18065.0,38725.0,-0.76,6.81,13.66,21.40,11.0,0.47,2.06,14.38,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.09,56.0,H-LC,5.91,215706.0,11206.0,30199.0,0.88,5.48,14.00,20.25,10.0,0.37,1.57,14.69,X40N,BTT,PHARMA
83,VBL,671.64,-16.58,38.0,H-LC,6.04,300689.0,-15153.0,127823.0,-0.71,-4.80,42.51,35.67,5.0,-0.12,2.19,8.39,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3986.49,68.0,M-SC,7.14,127381.0,-19987.0,71805.0,11.38,-13.56,56.37,35.16,236.0,-0.28,0.93,30.11,XY24,NTT,AUTO
58,ROUTE,2236.84,-46.24,51.0,H-SC,18.01,88602.0,-44464.0,137315.0,2.50,-33.42,154.98,69.78,118.0,-0.32,0.64,3.46,SR,ATH,IT
32,HINDZINC,730.22,20.84,63.0,M-LC,1.74,197164.0,-7912.0,120487.0,1.57,-3.86,61.11,54.89,52.0,-0.07,1.43,19.31,X5K,ATH,METALS
39,INFY,2275.00,-16.37,56.0,H-LC,9.45,325741.0,12853.0,158831.0,1.30,4.11,48.76,54.87,3.0,0.08,2.37,11.03,X40,BTT,IT
73,TATAMOTORS,1065.00,-18.21,62.0,M-LC,2.15,259364.0,-17159.0,127244.0,1.23,-6.21,49.06,39.81,54.0,-0.13,1.89,24.30,XY24,NTT,AUTO


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.0,38.16,47.0,H-SC,3.30,127500.0,-51571.0,117300.0,-2.76,-28.80,92.00,36.71,135.0,-0.44,0.93,26.51,XR,NTT,BANKS
27,GREENPANEL,537.0,244.96,72.0,M-SC,5.39,154037.0,-29041.0,108026.0,-2.02,-15.86,70.13,43.14,230.0,-0.27,1.12,44.89,XY24,NTT,MISC
77,TTKPRESTIG,770.0,100.85,51.0,M-SC,5.02,86532.0,-14245.0,14338.0,-1.87,-14.14,16.57,0.09,245.0,-0.99,0.63,13.68,OX40N,NTT,DURABLES
24,DABUR,735.0,-4.20,53.0,H-MC,6.13,208514.0,11486.0,78130.0,-1.74,5.83,37.47,45.49,102.0,0.15,1.52,19.63,XY24,BTT,FMCG
53,RAJESHEXPO,518.0,1834.34,63.0,L-SC,5.53,52775.0,-84402.0,84498.0,-1.65,-61.53,160.11,0.07,267.0,-1.00,0.38,31.25,OX40N,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,99.53,61.0,H-SC,12.57,126360.0,-16119.0,35646.0,-0.93,-11.31,28.21,13.70,163.0,-0.45,0.92,49.14,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,100.85,51.0,M-SC,5.02,86532.0,-14245.0,14338.0,-1.87,-14.14,16.57,0.09,245.0,-0.99,0.63,13.68,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,60.0,H-SC,2.00,224055.0,-45612.0,81937.0,0.95,-16.91,36.57,13.47,138.0,-0.56,1.63,13.32,XY24,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.14,146038.0,-29865.0,71880.0,0.13,-16.98,49.22,23.89,149.0,-0.42,1.06,26.12,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.01,53.0,H-MC,7.96,108196.0,-23639.0,65859.0,-1.01,-17.93,60.87,32.03,98.0,-0.36,0.79,21.19,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.40,294549.0,-51407.0,175257.0,0.30,-14.86,59.50,35.80,1.0,-0.29,2.14,4.35,X40,BTT,IT
39,INFY,2275.00,-16.37,56.0,H-LC,9.45,325741.0,12853.0,158831.0,1.30,4.11,48.76,54.87,3.0,0.08,2.37,11.03,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG
83,VBL,671.64,-16.58,38.0,H-LC,6.04,300689.0,-15153.0,127823.0,-0.71,-4.80,42.51,35.67,5.0,-0.12,2.19,8.39,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,11.97,250320.0,-11299.0,130517.0,-0.06,-4.32,52.14,45.57,7.0,-0.09,1.82,6.51,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,64.0,M-MC,6.95,227218.0,2256.0,165392.0,0.10,1.00,72.79,74.53,192.0,0.01,1.65,33.58,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.02,52.0,H-MC,3.95,188160.0,23490.0,44481.0,-0.28,14.26,23.64,41.28,89.0,0.53,1.37,23.39,X40N,ATH,AC
84,VOLTAS,1918.49,0.18,56.0,H-MC,2.97,210255.0,18513.0,77521.0,0.46,9.66,36.87,50.08,99.0,0.24,1.53,17.11,XY25,ATH,AC
24,DABUR,735.00,-4.20,53.0,H-MC,6.13,208514.0,11486.0,78130.0,-1.74,5.83,37.47,45.49,102.0,0.15,1.52,19.63,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,4.73,107912.0,5908.0,38017.0,0.09,5.79,35.23,43.06,122.0,0.16,0.79,23.90,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.23,114597.0,14093.0,16548.0,0.85,14.02,14.44,30.49,79.0,0.85,0.83,50.97,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.29,54.0,M-SC,1.42,100942.0,9444.0,69307.0,1.19,10.32,68.66,86.07,223.0,0.14,0.73,46.59,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.53,53.0,H-SC,12.25,137906.0,14570.0,119192.0,-0.40,11.81,86.43,108.45,119.0,0.12,1.00,36.81,AR,ATH,MISC
86,WIPRO,420.00,-11.52,53.0,M-LC,5.82,155800.0,4855.0,104604.0,-1.03,3.22,67.14,72.51,53.0,0.05,1.13,9.43,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.66,181961.0,2153.0,110268.0,0.18,1.20,60.60,62.52,88.0,0.02,1.32,35.69,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6700.00,51.0,L-SC,5.90,74603.0,-19207.0,95828.0,0.28,-20.47,128.45,81.67,269.0,-0.20,0.54,44.47,XR,NTT,CERAMICS
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.53,87072.0,-26477.0,66532.0,-0.70,-23.32,76.41,35.27,268.0,-0.40,0.63,113.11,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.35,44.0,H-SC,14.01,88167.0,-12796.0,107326.0,0.02,-12.67,121.73,93.63,148.0,-0.12,0.64,31.32,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,56.0,H-SC,13.13,94770.0,-3210.0,24811.0,0.00,-3.28,26.18,22.05,152.0,-0.13,0.69,37.76,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.23,114597.0,14093.0,16548.0,0.85,14.02,14.44,30.49,79.0,0.85,0.83,50.97,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3986.49,68.0,M-SC,7.14,127381.0,-19987.0,71805.0,11.38,-13.56,56.37,35.16,236.0,-0.28,0.93,30.11,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.53,87072.0,-26477.0,66532.0,-0.70,-23.32,76.41,35.27,268.0,-0.40,0.63,113.11,XR,NTT,HOTELS
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.23,114597.0,14093.0,16548.0,0.85,14.02,14.44,30.49,79.0,0.85,0.83,50.97,MH,ATH,METALS
71,TANLA,1963.11,-19.51,74.0,H-SC,9.29,215990.0,-54771.0,384721.0,0.24,-20.23,178.12,121.86,133.0,-0.14,1.57,68.96,AR,ATH,IT
27,GREENPANEL,537.00,244.96,72.0,M-SC,5.39,154037.0,-29041.0,108026.0,-2.02,-15.86,70.13,43.14,230.0,-0.27,1.12,44.89,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,43.51
2,50,73.83


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.50
LC    34.47
MC    22.99
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.99
X40      15.28
X40N     12.98
XR        9.99
XY25      9.74
AR        8.63
OX40N     8.18
X200      1.74
MH        1.72
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.36
H-LC    26.26
H-MC    20.05
M-SC    13.59
M-LC     7.15
M-MC     2.63
L-SC     1.55
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.41,-13.96,78.75
FMCG,11.83,-4.07,40.48
FINANCE,9.60,-16.57,62.23
BANKS,7.80,-11.99,63.54
MISC,6.07,-13.58,78.42
PAINTS,5.92,-13.51,38.91
ELECTRICAL,5.58,-7.47,66.70
AC,3.61,2.26,30.84
INSURANCE,3.38,-8.17,45.27


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2859993.0,22
AR,1310816.0,9
XR,1236773.0,13
X40,866666.0,10
X40N,630961.0,10
OX40N,521889.0,10
XY25,412159.0,7
SR,244641.0,2
X5K,120487.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353114.0,28
H-LC,1383750.0,15
M-SC,1378647.0,17
H-MC,1197859.0,15
M-LC,396268.0,5
M-MC,315257.0,2
L-SC,246858.0,3
L-MC,59959.0,1
L-LC,38296.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1040985.0      6
M-SC       XY24       790958.0      7
H-SC       AR         779482.0      5
           XR         683416.0      6
H-LC       X40        576716.0      5
H-MC       XY24       487087.0      4
H-LC       AR         369424.0      2
M-MC       XY24       315257.0      2
H-SC       X40N       296126.0      4
           OX40N      245951.0      4
           SR         244641.0      2
H-LC       X40N       224924.0      4
H-MC       X40        219811.0      4
           XY25       188375.0      2
L-SC       XR         162360.0      2
M-SC       AR         161910.0      2
M-LC       XY24       160640.0      2
M-SC       OX40N      125581.0      4
M-LC       X5K        120487.0      1
M-SC       XR         116166.0      2
           XY25       113893.0      1
H-MC       XR         110268.0      1
           X40N       109911.0      2
M-LC       XR         104604.0      1
H-LC       X200        86562.0      1
L-SC       OX40N       84498.0      1
M-SC       X40         70139.0      1
H-MC       OX40N       65859.0      1
H-LC       XY24        65066.0      1
H-SC       MH          62513.0      1
H-LC       XY25        61058.0      2
L-MC       XR          59959.0      1
L-LC       XY25        38296.0      1
H-MC       MH          16548.0      1
M-LC       XY25        10537.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
